<a href="https://colab.research.google.com/github/Nick088Official/Minecraft_Skin_Generator/blob/main/Minecraft_Skin_Generator_WEB_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minecraft Skin Generator WEB UI

**WARNING: WEB UI COULD RISK YOUR FREE TIER COLAB ACC**

Credits:
- Author of the AI Model: Cory Spencer <cory@monadical.com>
- Forked Improved Version & Ports: [Nick088](https://linktr.ee/Nick088)
- daroche (helping me fix the 3d model texture isue)
- [Brottweiler](https://gist.github.com/Brottweiler/483d0856c6692ef70cf90bf1a85ce364) (script to fix the 3d model texture)
- [not-holar](https://huggingface.co/not-holar) (made the rendering of the image in the web ui look pixelated like minecraft and have a checkered background)
- [meew](https://huggingface.co/spaces/meeww/Minecraft_Skin_Generator/blob/main/models/player_model.glb) (Minecraft Player 3d model)

In [ ]:
#@title Install
#@markdown If you get a restart runtime warning, you don't have to, just click cancel
!git clone https://github.com/Nick088Official/Minecraft_Skin_Generator.git
%cd Minecraft_Skin_Generator
!pip install -r Scripts/requirements_ui.txt
!apt update
!apt install -y imagemagick psmisc
from IPython.display import clear_output
clear_output()
print("Installed!")

In [ ]:
#@title Run Inference UI

#@markdown The type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudflare", "LocalTunnel", "Horizon"]

#@markdown Also when using Ngrok ,Cloudflare, LocalTunnel or Horizon, you need to wait for the Local URL to appear, and only after that click on the Public URL which is above.

#@markdown Use the following option **only if you chose Ngrok** as the Tunnel:

#@markdown You can get the Ngrok Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_authtoken = "" #@param {type:"string"}


#@markdown Use the following option **only if you chose Horizon** as the Tunnel:

#@markdown You can get the Horizon ID here: https://hrzn.run/dashboard/ , login, on the 2nd step, it shows an `hrzn login YOUR_ID`, you need to copy that id.

horizon_id = "" #@param {type:"string"}


if Tunnel == "Gradio":
  share_flag = "--share"
elif Tunnel == "Ngrok":
  share_flag = ""
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_authtoken)
  http_tunnel = ngrok.connect(7860, bind_tls=True)
  print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudflare":
  share_flag = ""
  # download cloudflare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  !nohup cloudflared tunnel --url localhost:7860 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"Cloudflare Tunnel Public URL: {cloudflare_url[0]}")
elif Tunnel == "LocalTunnel":
  share_flag = ""
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()
  print(f"LocalTunnel Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  print(f'LocalTunnel Password: {endpoint_ip}')
elif Tunnel == "Horizon":
  share_flag = ""
  # install
  !npm install -g @hrzn/cli
  import time
  # login
  !hrzn login $horizon_id
  # run horizon
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('hrzn tunnel http://localhost:7860 >> url.txt 2>&1 &')

  time.sleep(4)

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url.txt
      tunnel_url = tunnel_url[0]

  clear_output()

  print(f"Horizon Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m")


# kills previously running processes
!fuser -k 7860/tcp


command = f"python Scripts/Run_Inference_UI.py {share_flag}"
!{command}